<a href="https://colab.research.google.com/github/SanskarGithub07/Fine-Tuning-LLMs-for-Spam-Detection-and-Emotion-Classification/blob/main/LLM_Project_2_Sanskar_Sugandhi_2210110898.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise-1

In [ ]:
!pip install tiktoken

In [ ]:
!pip install -U fsspec datasets

In [ ]:
!pip install tiktoken transformers datasets

In [ ]:
# Download the data
import urllib.request
import zipfile
import os
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

    # Downloading the file
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    # Unzipping the file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

In [ ]:
# Load and explore the dataset
import pandas as pd

df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

In [ ]:
print(df["Label"].value_counts())

In [ ]:
# Create a balanced dataset
def create_balanced_dataset(df):
    # Count the instances of "spam"
    num_spam = df[df["Label"] == "spam"].shape[0]

    # Randomly sample "ham" instances to match the number of "spam" instances
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)

    # Combine ham "subset" with "spam"
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])

    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

# Map labels to integers
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})
balanced_df

In [ ]:
# Split the dataset
def random_split(df, train_frac, validation_frac):
    # Shuffle the entire DataFrame
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Calculate split indices
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # Split the DataFrame
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)
# Save the split datasets
train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

In [ ]:
# Import SmolLM2 tokenizer from HuggingFace
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# Load the SmolLM2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# If tokenizer doesn't have a padding token set, use the EOS token as padding token. Crucial when batching inputs of different lengths. Pad shorter sequences to match the length of longer sequence
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text, add_special_tokens=True)
            for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length() #Automatically calculate the max length of any tokenized sequence
        else:
            self.max_length = max_length
            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [tokenizer.pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

In [ ]:
# Create datasets
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(f"Maximum sequence length in training set: {train_dataset.max_length}")

val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

In [ ]:
# Create data loaders
batch_size = 8
num_workers = 0

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [ ]:
print("Train loader:")
for input_batch, target_batch in train_loader:
    break

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions:", target_batch.shape)

print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

In [ ]:
from transformers import AutoModelForCausalLM

# Load SmolLM2-135M model
model_name = "HuggingFaceTB/SmolLM2-135M"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a model class that wraps the pretrained model and adds a classification head
class SmolLM2Classifier(torch.nn.Module):
    def __init__(self, base_model, num_classes=2):
        super().__init__()
        self.base_model = base_model
        self.config = base_model.config

        # Freeze all parameters (gradients are not calculated for those params)
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Get embedding dimension
        self.hidden_size = self.config.hidden_size

        # Add a classification head
        self.classifier = torch.nn.Linear(self.hidden_size, num_classes)

        # Unfreeze the last transformer block and layer norm
        for param in self.base_model.model.layers[-1].parameters():
            param.requires_grad = True

        for param in self.base_model.model.norm.parameters():
            param.requires_grad = True

    def forward(self, input_ids):
        # Get the output from the base model
        outputs = self.base_model(input_ids, output_hidden_states=True)

        # Get the last hidden state
        last_hidden_state = outputs.hidden_states[-1]

        # Apply the classifier to get logits
        logits = self.classifier(last_hidden_state)

        return logits

In [ ]:
# Initialize the classification model
model = SmolLM2Classifier(base_model)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

In [ ]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  # Logits of last output token
            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break

    return correct_predictions / num_examples

def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  # Logits of last output token
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break

    return total_loss / num_batches

In [ ]:
# Check initial performance
with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Initial training loss: {train_loss:.3f}")
print(f"Initial validation loss: {val_loss:.3f}")
print(f"Initial test loss: {test_loss:.3f}")

train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, model, device, num_batches=10)

print(f"Initial training accuracy: {train_accuracy*100:.2f}%")
print(f"Initial validation accuracy: {val_accuracy*100:.2f}%")
print(f"Initial test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    # Initialize lists to track losses and examples seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            examples_seen += input_batch.shape[0] # Track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen

In [ ]:
import time

start_time = time.time()

torch.manual_seed(123)

# Create optimizer - note we're only optimizing the classifier parameters and unfrozen layers
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=5e-5,
    weight_decay=0.1 #Regularization term penalizes large weights prevents overfitting
)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import matplotlib.pyplot as plt

def plot_values(epochs_seen, examples_seen, train_values, val_values, label="loss"):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_values, label=f"Training {label}")
    ax1.plot(epochs_seen, val_values, linestyle="-.", label=f"Validation {label}")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel(label.capitalize())
    ax1.legend()

    # Create a second x-axis for examples seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(examples_seen, train_values, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Examples seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig(f"smollm2_{label}-plot.pdf")
    plt.show()

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))

plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_accs))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_accs))

plot_values(epochs_tensor, examples_seen_tensor, train_accs, val_accs, label="accuracy")

In [ ]:
# Calculate final performance on full datasets
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)

print(f"Final training accuracy: {train_accuracy*100:.2f}%")
print(f"Final validation accuracy: {val_accuracy*100:.2f}%")
print(f"Final test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
def classify_text(text, model, tokenizer, device, max_length=None):
    model.eval()

    # Tokenize the input text
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Truncate if needed
    if max_length is not None:
        input_ids = input_ids[:max_length]

    # Pad if needed
    if max_length is not None and len(input_ids) < max_length:
        input_ids += [tokenizer.pad_token_id] * (max_length - len(input_ids))

    # Convert to tensor and add batch dimension
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0)

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token

    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return "spam" if predicted_label == 1 else "not spam"

In [ ]:
text_examples = [
    "You are a winner you have been specially selected to receive $1000 cash or a $2000 award.",
    "Hey, just wanted to check if we're still on for dinner tonight? Let me know!",
    "URGENT! You have won a 1-week FREE membership in our £100,000 Prize Jackpot! Text the word: CLAIM to 81010",
    "I'll be there in about 15 minutes. See you soon.",
    "Congratulations! You've been selected for a free iPhone 13. Click here to claim now!",
    "Can you pick up some milk on your way home? Thanks."
]

for text in text_examples:
    result = classify_text(text, model, tokenizer, device, max_length=train_dataset.max_length)
    print(f"Text: {text}\nClassification: {result}\n")

In [ ]:
# Save the model for later use
torch.save(model.state_dict(), "smollm2_spam_classifier.pth")

print("Model saved successfully! You can reload it with:")
print("""
# Load the saved model
model = SmolLM2Classifier(base_model)  # Initialize the model structure first
model.load_state_dict(torch.load("smollm2_spam_classifier.pth"))
model.eval()
""")

# Exercise-2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Load the emotion dataset
print("Loading emotion dataset...")
dataset = load_dataset("emotion")
print(dataset)

In [ ]:
# Get dataset statistics
def get_dataset_stats(dataset):
    # Count occurrences of each label in train set
    label_counts = {}
    for item in dataset["train"]:
        label = item["label"]
        label_counts[label] = label_counts.get(label, 0) + 1

    # Sort labels by frequency
    sorted_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)

    # Get label names
    label_names = dataset["train"].features["label"].names
    print("Label mapping:", {i: name for i, name in enumerate(label_names)})

    # Print statistics
    print("\nLabel distribution in training set:")
    for label_id, count in sorted_labels:
        print(f"{label_names[label_id]}: {count} samples ({count/len(dataset['train'])*100:.2f}%)")

    return label_names, sorted_labels

label_names, sorted_labels = get_dataset_stats(dataset)
label_names, sorted_labels

In [ ]:
# Select top 4 emotions for our classification task
top_emotions = [label_id for label_id, _ in sorted_labels[:4]]
print(f"\nWe'll use these 4 emotions for classification: {[label_names[i] for i in top_emotions]}")

In [ ]:
# Filter dataset to only include the top 4 emotions
def filter_top_emotions(example):
    return example["label"] in top_emotions

filtered_train = dataset["train"].filter(filter_top_emotions)
filtered_validation = dataset["validation"].filter(filter_top_emotions)
filtered_test = dataset["test"].filter(filter_top_emotions)

print(f"Filtered train set: {len(filtered_train)} samples")
print(f"Filtered validation set: {len(filtered_validation)} samples")
print(f"Filtered test set: {len(filtered_test)} samples")

# Create label mapping for the filtered dataset (0-3)
emotion_id_mapping = {old_id: new_id for new_id, old_id in enumerate(top_emotions)}

# Map old label IDs to new consecutive label IDs
def map_labels(example):
    example["new_label"] = emotion_id_mapping[example["label"]]
    return example

filtered_train = filtered_train.map(map_labels)
filtered_validation = filtered_validation.map(map_labels)
filtered_test = filtered_test.map(map_labels)

filtered_train, filtered_test, filtered_validation

In [ ]:
# Create balanced dataset
def create_balanced_dataset(dataset):
    # Find the minimum count among all classes
    label_counts = {}
    for item in dataset:
        label = item["new_label"]
        label_counts[label] = label_counts.get(label, 0) + 1

    min_count = min(label_counts.values())

    # Group examples by label
    grouped_examples = {label: [] for label in label_counts.keys()}
    for item in dataset:
        grouped_examples[item["new_label"]].append(item)

    # Sample min_count examples from each class
    balanced_examples = []
    for label, examples in grouped_examples.items():
        sampled_examples = examples[:min_count]  # Take first min_count examples
        balanced_examples.extend(sampled_examples)

    # Convert to pandas DataFrame
    balanced_df = pd.DataFrame({
        "Text": [item["text"] for item in balanced_examples],
        "Label": [item["new_label"] for item in balanced_examples]
    })

    return balanced_df

# Create balanced datasets
balanced_train_df = create_balanced_dataset(filtered_train)
balanced_val_df = create_balanced_dataset(filtered_validation)
balanced_test_df = create_balanced_dataset(filtered_test)

print(f"Balanced train set: {len(balanced_train_df)} samples")
print(f"Balanced validation set: {len(balanced_val_df)} samples")
print(f"Balanced test set: {len(balanced_test_df)} samples")

# Display class distribution in balanced train set
print(balanced_train_df["Label"].value_counts())

In [ ]:
# Save the balanced datasets
balanced_train_df.to_csv("train.csv", index=None)
balanced_val_df.to_csv("validation.csv", index=None)
balanced_test_df.to_csv("test.csv", index=None)

# Get emotion names for the new label mapping
new_label_names = [label_names[old_id] for old_id in top_emotions]
print(f"New label mapping: {dict(enumerate(new_label_names))}")

In [ ]:
# Load the SmolLM2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# If tokenizer doesn't have a padding token set, use the EOS token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

class EmotionDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text, add_special_tokens=True)
            for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [tokenizer.pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

# Create datasets
train_dataset = EmotionDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(f"Maximum sequence length in training set: {train_dataset.max_length}")

val_dataset = EmotionDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

test_dataset = EmotionDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

# Create data loaders
batch_size = 16  # Increased batch size since emotion texts are typically shorter
num_workers = 0

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

# Verify dataset and loader dimensions
print("Train loader:")
for input_batch, target_batch in train_loader:
    break

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions:", target_batch.shape)

print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

In [ ]:
# Load SmolLM2-135M model
model_name = "HuggingFaceTB/SmolLM2-135M"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a model class that wraps the pretrained model and adds a classification head
class SmolLM2Classifier(torch.nn.Module):
    def __init__(self, base_model, num_classes=4):  # Changed to 4 classes for emotion
        super().__init__()
        self.base_model = base_model
        self.config = base_model.config

        # Freeze all parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Get embedding dimension
        self.hidden_size = self.config.hidden_size

        # Add a classification head
        self.classifier = torch.nn.Linear(self.hidden_size, num_classes)

        # Unfreeze the last transformer block and layer norm
        for param in self.base_model.model.layers[-1].parameters():
            param.requires_grad = True

        for param in self.base_model.model.norm.parameters():
            param.requires_grad = True

    def forward(self, input_ids):
        # Get the output from the base model
        outputs = self.base_model(input_ids, output_hidden_states=True)

        # Get the last hidden state
        last_hidden_state = outputs.hidden_states[-1]

        # Apply the classifier to get logits
        logits = self.classifier(last_hidden_state)

        return logits

# Initialize the classification model
model = SmolLM2Classifier(base_model, num_classes=4)  # 4 emotion classes

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

In [ ]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  # Logits of last output token
            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break

    return correct_predictions / num_examples

def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  # Logits of last output token
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break

    return total_loss / num_batches

In [ ]:
# Check initial performance
with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Initial training loss: {train_loss:.3f}")
print(f"Initial validation loss: {val_loss:.3f}")
print(f"Initial test loss: {test_loss:.3f}")

train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, model, device, num_batches=10)

print(f"Initial training accuracy: {train_accuracy*100:.2f}%")
print(f"Initial validation accuracy: {val_accuracy*100:.2f}%")
print(f"Initial test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    # Initialize lists to track losses and examples seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            examples_seen += input_batch.shape[0] # Track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen

In [ ]:
# Train the model
import time

start_time = time.time()

torch.manual_seed(123)

# Create optimizer - note we're only optimizing the classifier parameters and unfrozen layers
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=5e-5,
    weight_decay=0.1
)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import matplotlib.pyplot as plt

def plot_values(epochs_seen, examples_seen, train_values, val_values, label="loss"):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_values, label=f"Training {label}")
    ax1.plot(epochs_seen, val_values, linestyle="-.", label=f"Validation {label}")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel(label.capitalize())
    ax1.legend()

    # Create a second x-axis for examples seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(examples_seen, train_values, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Examples seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig(f"smollm2_emotion_{label}-plot.pdf")
    plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))

plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

epochs_tensor = torch.linspace(0, num_epochs, len(train_accs))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_accs))

plot_values(epochs_tensor, examples_seen_tensor, train_accs, val_accs, label="accuracy")

In [ ]:
# Calculate final performance on full datasets
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)

print(f"Final training accuracy: {train_accuracy*100:.2f}%")
print(f"Final validation accuracy: {val_accuracy*100:.2f}%")
print(f"Final test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
# Create a confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

def create_confusion_matrix(model, data_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for input_batch, target_batch in data_loader:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)
            logits = model(input_batch)[:, -1, :]
            predicted_labels = torch.argmax(logits, dim=-1)

            all_preds.extend(predicted_labels.cpu().numpy())
            all_labels.extend(target_batch.cpu().numpy())

    # Create confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig("emotion_confusion_matrix.pdf")
    plt.show()

    return cm

# Create confusion matrix for test set
cm = create_confusion_matrix(model, test_loader, device, new_label_names)

In [ ]:
def classify_emotion(text, model, tokenizer, device, max_length=None, emotion_names=None):
    model.eval()

    # Tokenize the input text
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Truncate if needed
    if max_length is not None:
        input_ids = input_ids[:max_length]

    # Pad if needed
    if max_length is not None and len(input_ids) < max_length:
        input_ids += [tokenizer.pad_token_id] * (max_length - len(input_ids))

    # Convert to tensor and add batch dimension
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0)

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token

    # Get probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the predicted emotion and confidence
    emotion = emotion_names[predicted_label] if emotion_names else str(predicted_label)
    confidence = probs[0][predicted_label].item()

    return emotion, confidence

# Test the classifier with example messages
text_examples = [
    "I feel so happy today, everything is going well!",
    "I can't believe they would do this to me, I'm so angry right now.",
    "My heart is broken, I can't stop crying after what happened.",
    "I love spending time with my family, they bring me so much joy.",
    "That horror movie was so scary, I couldn't sleep all night.",
    "I miss you so much, can't wait to see you again."
]

print("Emotion Classification Results:")
print("-" * 60)
for text in text_examples:
    emotion, confidence = classify_emotion(text, model, tokenizer, device,
                                          max_length=train_dataset.max_length,
                                          emotion_names=new_label_names)
    print(f"Text: {text}")
    print(f"Predicted emotion: {emotion} (confidence: {confidence:.2f})")
    print("-" * 60)

In [ ]:
# Save the model for later use
torch.save(model.state_dict(), "smollm2_emotion_classifier.pth")

print("Model saved successfully! You can reload it with:")
print("""
# Load the saved model
model = SmolLM2Classifier(base_model, num_classes=4)  # Initialize the model structure first
model.load_state_dict(torch.load("smollm2_emotion_classifier.pth"))
model.eval()
""")

# Print final information about the dataset and model
print("\nFinal model summary:")
print(f"Dataset: Emotion classification (subset of 4 emotions)")
print(f"Classes: {new_label_names}")
print(f"Model: SmolLM2-135M with classification head")
print(f"Final test accuracy: {test_accuracy*100:.2f}%")